In [9]:
import heapq

""" precise belt placement """

def heuristic(a, b):
    """Calculate the Manhattan distance between a and b."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(start, goal, mask):
    """A* algorithm implementation."""
    # The open list is a priority queue (heap)
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while open_list:
        current_cost, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current is not None:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        for next, cost in neighbours(current, mask):
            new_cost = cost_so_far[current] + cost  # Assuming uniform cost
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                heapq.heappush(open_list, (priority, next))
                came_from[next] = current

    return None  # Path not found


def is_valid_move(node, next_node, dx, dy, mask):
    if dx != 0:
        for ddx in range(min(0, dx), max(0, dx)):
            check_node = node[0] + ddx, node[1]
            if not is_valid_check_node(check_node, mask, 0):
                return False
    if dy != 0:
        for ddy in range(min(0, dy), max(0, dy)):
            check_node = node[0], node[1] + ddy
            if not is_valid_check_node(check_node, mask, 1):
                return False
    return True

def is_valid_check_node(check_node, mask, index):
    return (check_node in mask and (not mask[check_node] or 
            mask[check_node][0].startswith("underground") and 
            mask[check_node][1][index] == 0))

def neighbours(node, mask):
    """Generate neighbors for a node considering the mask."""
    directions = [
        ("belt", (1, 0)),
        ("belt", (-1, 0)),
        ("belt", (0, 1)),
        ("belt", (0, -1)),
        ("underground", (2, 0)),
        ("underground", (3, 0)),
        ("underground", (4, 0)),
        ("underground", (-2, 0)),
        ("underground", (-3, 0)),
        ("underground", (-4, 0)),
        ("underground", (0, 2)),
        ("underground", (0, 3)),
        ("underground", (0, 4)),
        ("underground", (0, -2)),
        ("underground", (0, -3)),
        ("underground", (0, -4)),
    ]

    kind_cost = {"belt": 1, "underground": 10}
    result = []

    for kind, (dx, dy) in directions:
        next_node = (node[0] + dx, node[1] + dy)
        ok = kind != "underground" or is_valid_move(node, next_node, dx, dy, mask)

        if next_node in mask and not mask[next_node] and ok:
            result.append((next_node, kind_cost[kind]))
            
    return result

from itertools import product

# Example usage
start = (0, 0)
goal = (3, 3)

# True indicates an impassable area
mask = {(x, y): False for x, y in product(range(4), repeat=2)}


mask[0, 1] = ("underground", (1, 0))
mask[0, 3] = ("underground", (-1, 0))

path = a_star(start, goal, mask)
print("Path:", path)

TypeError: 'bool' object is not subscriptable